In [112]:
from nltk.compat import remove_accents
from yaml.tokens import Token
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from collections import OrderedDict
import json
import os.path

# import nltk
# nltk.download('punkt')

ps = PorterStemmer()
class BooleanRetrievalModel:
    dict = OrderedDict()    #Key: string(term) Value: List(specifically docIds in which that term occured)
    posi = OrderedDict()
    NumberOfDocs = 50
    stopwordslist = []
    def __init__(self):
       self.createPositionalIndex()
      #  self.createInvertedIndex()
      #  self.donothing()
      #  self.write()
       self.read()
      #  self.writeToFile()
    
    # def writeToFile(self):
    #   f = open("/content/new.txt" , "w")
      # for K , V in self.posi.items():
      #   f.write(K + "\n")
      #   # f.write(V + "\n")
      #   for K1 , V1 in V.items():
      #     f.write(str(K1) + " ")
      #     f.write(str(V1))
      #   f.write("\n")

    # def read():

    def createPositionalIndex(self):
      sf = open("/content/stopwords.txt" , "r")
      for line in sf:
        for sword in line.split():
          self.stopwordslist.append(sword)
      sf.close()
      for i in range(self.NumberOfDocs):
        f = open("/content/" + str(i + 1) + ".txt")

        index = 1

        for line in f:
          for word in line.split():
            if word not in (self.stopwordslist):
              word = word.strip(".,?:)(%'1234567890#$!{}")
              word = word.replace("/" , " ")
              word = word.replace("-" , " ")
              tokenizedword = word_tokenize(word)
              for word in tokenizedword:
                word = ps.stem(word)
                if( word in self.posi):
                  x = self.posi[word]
                  if i + 1 not in x.keys():
                    x[i + 1] = OrderedDict()
                    x[i + 1] = [index]
                  else:
                    x[i + 1].append(index)
                else:
                  self.posi[word] = OrderedDict()
                  self.posi[word][i + 1] = [index]
                index += 1
            else:
              index += 1
        f.close()
      print(self.posi)
      print(type(self.posi))
      self.createInvertedIndex()


    def createInvertedIndex(self):
      for K , V in self.posi.items():
        self.dict[K] = list(V.keys())
      print(self.dict)













      # sf = open("/content/stopwords.txt" , "r")
      # for line in sf:
      #    for sword in line.split():
      #      self.stopwordslist.append(sword)
      # i = 1
      # sf.close()
      # for i in range(self.NumberOfDocs):
      #      # print(i + 1)
      #      f = open( "/content/" + str(i + 1) + ".txt" , "r")
      #      for line in f:
      #        for word in line.split():
      #          if word not in (self.stopwordslist):
      #            word = word.strip(".,?:)(%'1234567890#$!")
      #            word = word.replace("/" , " ")
      #            word = word.replace("-" , " ")

      #            tokenizedword = word_tokenize(word)
      #            for word in tokenizedword:
      #              word = ps.stem(word)
      #            if word not in self.dict:
      #              self.dict[word] = []
      #              self.dict[word].append(i + 1)
      #            else:
      #              if i + 1 not in self.dict[word]:
      #                self.dict[word].append(i + 1)
      #      f.close()
      #  # self.dict = sorted(self.dict)
      # print(type(self.dict))
      # print(self.dict)

    
        
    

        
      
    def ProximityQuery(self , term1 , term2 , k):
      p1 = OrderedDict()
      p2 = OrderedDict()
      term1 = ps.stem(term1)
      term2 = ps.stem(term2)
      if( term1 in self.posi):
        p1 =  self.posi[term1]
      if( term2 in self.posi):
        p2 =  self.posi[term2]
      print("Sending Doc Ids")
      print(p1.keys())
      print(p2.keys())
      result = self.PositionalIntersect( list(p1.keys()) , list(p2.keys()),k , term1 , term2)


    def PositionalIntersect(self , p1 , p2 , k , term1 , term2):
      i , j = 0 , 0
      result = []
      while( i < len(p1) and j < len(p2)):
        x = p1[i]
        y = p2[j]
        print(x)
        print(y)
        if(x == y):
          pp1 = self.posi[term1][x]
          pp2 = self.posi[term2][y]
          i1 , j1  = 0 , 0
          flag = False
          print(pp1)
          print(pp2)

          while ( i1 < len(pp1) ):
            while(j1 < len(pp2) ):
              if(abs( pp1[i1] - pp2[j1]) <= k):
                # print(p1[i])
                result.append(p1[i])
                flag = True
                break
              j1 += 1
            if(flag):
              break
            i1 += 1

          i += 1
          j += 1
          
        elif (x <= y):
          i += 1
        else:
          j += 1
      print("Result")
      print(result)
      return result
      
    def Intersection(self , postinglist1 , postinglist2):
      i , j = 0 , 0
      result = []
      while i != len(postinglist1) and j != len(postinglist2):
        if postinglist1[i] == postinglist2[j]:
          result.append(postinglist1[i])
          i += 1 
          j += 1
        elif postinglist1[i] < postinglist2[j]:
          i += 1
        else:
          j += 1 
      return result


    def Merge(self , postinglist1 , postinglist2):
      result = list(set(postinglist1 + postinglist2))
      return result
    def OrQuery(self , term1 , term2):
      p1 = []
      p2 = []
      if(term1 in self.dict[term1]):
        p1 = self.dict[term1]
      if(term2 in self.dict[term2]):
        p2 = self.dict[term2]
      return self.Merge(term1 , term2)
    def AndQuery(self , term1 , term2):
      term1 = ps.stem(term1)
      term2 = ps.stem(term2)
      postinglist1 = []
      postinglist2 = []
      if term1 in self.dict:
        postinglist1 = self.dict[term1]
      if term2 in self.dict:
        postinglist2 = self.dict[term2]
      return self.Intersection(postinglist1 , list(postinglist2))
    def Complement(self , term):
      postinglist = []
      term = ps.stem(term)
      if term in self.dict:
        postinglist = self.dict[term]
      print(postinglist)
      return self.NotQuery(postinglist)

    def NotQuery(self , postinglist1):
      totalDocIds = list(range (1 ,1 + self.NumberOfDocs))
      result = [item for item in totalDocIds if item not in postinglist1]

      return result

    def Query(self , query):
      x = query.split(' ')
      k = query[-1]
      if(x[-1] == str("/" + k)):
        self.ProximityQuery(x[0] , x[1] , int(k))
      else:
        self.preprocessQuery( query)



    def preprocessQuery(self , query):
      i = 0
      print("in pre processing")
      terms = []
      operators = []
      tokenizedquery = word_tokenize(query)
      while (i < len(tokenizedquery)):
        if(tokenizedquery[i] == "and"):
          operators.append("and")
          i += 1
        elif(tokenizedquery[i] == "or"):
          operators.append("or")
          i += 1
        elif(tokenizedquery[i] == "not"):
          terms.append(tokenizedquery[i + 1])
          terms.append("not")
          i += 2
        else:
          terms.append(tokenizedquery[i])
          terms.append("zz")
          i += 1        
      self.process(terms , operators)
    def allpostings(self , terms):
      postings = []
      i = 0
      while ( terms ):
        term = terms.pop(0)
        term = ps.stem(term)
        if(terms[0] == "not"):
          postings.append(self.Complement(term))
        else:
           print("term: " , term)
           postings.append(self.postingsofterm(term))
        terms.pop(0)
      return postings
    def postingsofterm(self , term):
      ret = []
      if(term in self.dict):
        ret = self.dict[term]
      print("posting list of " + term + ":" , ret)
      return ret

      


    def processinvertedindex(self , p1 , p2 , operator):
      result = []
      if(operator == 'and'):
         result = self.Intersection(p1 , p2) 
      elif(operator == 'or'):
        result = self.Merge(p1 , p2)
      return result
    def process(self , terms , operators):
      print("in porcess")
      allpostings = self.allpostings(terms)
      print(type(allpostings))
      print(operators)
      while( operators):
        operation = operators.pop(0)
        p1 = allpostings.pop(0)
        # allpostings.pop(0)
        p2 = allpostings.pop(0)
        allpostings.insert(0 , self.processinvertedindex(p1 , p2 , operation))
      matchedDocs = allpostings.pop(0)
      print(sorted(self.dict))
      print(matchedDocs)
    def write(self):
      towrite = (json.dumps(list(self.posi.items())))
      f= open("/content/n.json" , "w")
      f.write(towrite)
      f.close()
    def read(self):
      if os.path.isfile("Index.json"):
        f = open("/content/Index.json" , "r")
        toread = json.loads(f.read())
        i = 0
        x = self.posi
        while( i < len(toread)):
          y = toread[i][1]
          z = {int(K) : V for K,V in y.items()}
          x[toread[i][0]] = OrderedDict(z)    ## 00 is term 01 is dict key = doc id val = list
          i += 1
        f.close()
      else:
        self.createPositionalIndex()
        self.write()
     
          # word = word.strip(".,?:)(%'1234567890#$!{}")
          #     word = word.replace("/" , " ")
          #     word = word.replace("-" , " ")
          #     tokenizedword = word_tokenize(word)
          #     for word in tokenizedword:
          #       word = ps.stem(word)
          #       if( word in self.posi):
          #         x = self.posi[word]
          #         if i + 1 not in x.keys():
          #           x[i + 1] = OrderedDict()
          #           x[i + 1] = [index]
          #         else:
          #           x[i + 1].append(index)
          #       else:
          #         self.posi[word] = OrderedDict()
          #         self.posi[word][i + 1] = [index]
          #       index += 1
          #   else:
          #     index += 1
     
     
     
     
    def donothing(self):
      towrite = json.dumps(list(self.posi.items()))
      print(type(towrite))
      print(towrite)


      toread = json.loads(towrite)
      print(toread[0])
      print(type(toread))
      print(type(toread[0][1]))
b = BooleanRetrievalModel()
# b.ProximityQuery( "ensemble" , "model" , 5
b.Query("text and classification")
# print(b.("pattern"))

OrderedDict([('ensembl', OrderedDict([(1, [1, 13, 49, 66]), (2, [15, 85, 113]), (3, [3, 20, 63, 81, 100]), (5, [135]), (32, [208, 233])])), ('statist', OrderedDict([(1, [2, 10, 18]), (14, [68]), (15, [4, 6, 19, 120]), (17, [65]), (41, [20, 167]), (42, [45]), (43, [138])])), ('heurist', OrderedDict([(1, [4, 15, 97])])), ('model', OrderedDict([(1, [5, 21, 58, 94, 125]), (2, [94]), (3, [61, 132]), (4, [25, 108]), (5, [9, 34, 98, 129, 133, 137, 141]), (9, [157]), (10, [50, 65, 124]), (11, [112]), (13, [133]), (16, [80, 98, 109]), (18, [78]), (19, [54, 78]), (20, [27]), (22, [40]), (29, [33, 96]), (32, [10]), (34, [163]), (35, [46, 57, 82, 122, 154]), (36, [3, 19, 122, 189]), (37, [111, 222, 282]), (39, [67]), (42, [18, 46, 54, 81, 185]), (43, [139]), (46, [1, 56, 69, 141, 162, 180])])), ('unsupervis', OrderedDict([(1, [7, 43]), (15, [16]), (27, [53]), (31, [141])])), ('word', OrderedDict([(1, [8, 11, 16, 19, 45]), (9, [98, 147]), (27, [10, 17, 61]), (46, [169]), (48, [13, 164])])), ('align

In [ ]:
dict = {}
dict["mateen"] = 4
print(type(dict))


<class 'dict'>


# New Section

In [ ]:
!pip install nltk